# Starbucks Geospatial Customer Analytics


The goal of this analysis is to conduct customer analytics on 34 Starbucks locations in Atlanta, GA in order to determine the probability that a customer from a certain neighborhood in Atlanta will visit a given Starbucks. We will be implementing a Huff Model supported by Principal Component Analysis (PCA) to determine the probabilities. From this Huff Model, we will create an interactive heat map that the user can interact with to determine more about the area.

## What is a Huff Model?

The **Huff model** is a spatial interaction model used to estimate the probability that a consumer will choose a specific location (e.g., a store) over competing alternatives.

It combines two key concepts:
- **Attractiveness**: The inherent appeal of a location, which might be based on size, features, convenience, or amenities.
- **Distance decay**: The idea that as the distance (or travel time) to a location increases, the likelihood of a consumer choosing that location decreases.

### Formula
$$
P_{ij} = \frac{A_j^\lambda / D_{ij}^\beta}{\sum_{k=1}^{n} A_k^\lambda / D_{ik}^\beta}
$$

Where:
- $P_{ij}$: Probability that consumer in zone $i$ visits store $j$
- $A_j$: Attractiveness of store $j$
- $D_{ij}$: Distance (or travel time) from neighborhood $i$ to store $j$
- $\beta$: Distance decay parameter (typically > 1)
- $n$: Total number of competing stores

### Why Use It?

The Huff model is useful for:
- Estimating market share
- Defining trade areas
- Site selection and competitive analysis

We’ll use this model to estimate the likelihood of people in different neighborhoods around Atlanta choosing a specific Starbucks location over others.

In [1]:
# Core packages
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import shape
from shapely.geometry import mapping
import numpy as np

# Machine Learning (PCA)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Mapping
import folium
from folium import Map, Marker
from folium.plugins import MarkerCluster
from folium.features import CustomIcon
import matplotlib.cm as cm
import matplotlib.colors as colors

# Dataset Schemas and Summaries (With Descriptions)

### Starbucks Locations
- **Description**: List of Starbucks locations in Atlanta, GA. 
- **Source**: ArcGIS Online
- **Rows**: 34
- **Columns**:

| Column | Type | Description |
|--------|------|-------------|
| `OBJECTID` | `int64` | Unique row identifier |
| `Brand` | `object` | Brand name (e.g., Starbucks) |
| `Store_Numb` | `object` | Store number (internal ID) |
| `Store_Name` | `object` | Name of the Starbucks location |
| `Ownership_` | `object` | Ownership model (Company-owned, Licensed) |
| `Street_Add` | `object` | Street address |
| `City` | `object` | City |
| `State_Prov` | `object` | State or Province |
| `Country` | `object` | Country |
| `Postcode` | `object` | Postal code |
| `Phone_Numb` | `object` | Phone number |
| `Timezone` | `object` | Timezone |
| `Longitude` | `float64` | Longitude coordinate |
| `Latitude` | `float64` | Latitude coordinate |


### Point of Interest Data
- **Description**: Points of Interest (Schools, Universities, Supermarkets, Hospitals) in Atlanta
- **Source**: OpenStreetMap
- **Rows**: 161
- **Columns**:

| Column | Type | Description |
|--------|------|-------------|
| `FID` | `int64` | Feature ID |
| `osm_id` | `object` | OpenStreetMap identifier |
| `code` | `int64` | Numeric POI classification |
| `fclass` | `object` | Feature class (e.g., cafe, restaurant) |
| `name` | `object` | Name of the POI or business |


### Georgia Dept. of Transportation Traffic
- **Description**: Traffic counts for roads in Atlanta
- **Source**: Georgia Department of Transportation
- **Rows**: 2184
- **Columns**:

| Column | Type | Description |
|--------|------|-------------|
| `OBJECTID` | `int64` | Unique row identifier |
| `Station_ID` | `object` | Traffic station identifier |
| `Functional` | `object` | Functional class of road (e.g., highway, arterial) |
| `LatLong` | `object` | Latitude and Longitude (combined string) |
| `Lat` | `float64` | Latitude of the station |
| `Long` | `float64` | Longitude of the station |
| `AADT` | `int64` | Average Annual Daily Traffic |


### Competition
- **Description**: Competitors (Fast Food, Cafes) in Atlanta
- **Source**: OpenStreetMap
- **Rows**: 835
- **Columns**:

| Column | Type | Description |
|--------|------|-------------|
| `OBJECTID` | `int64` | Unique row identifier |
| `access` | `object` | Accessibility information (e.g., yes, private) |
| `amenity` | `object` | Type of amenity (e.g., cafe, fast_food) |
| `brand` | `object` | Business brand name |
| `cuisine` | `object` | Type of cuisine served |
| `name` | `object` | Name of the POI or business |
| `opening_hours` | `object` | Opening hours of the business |


### Demographics
- **Description**: Demographic information about neighborhoods in Atlanta
- **Source**: City of Atlanta OpenGIS
- **Rows**: 248
- **Columns**:

| Column | Type | Description |
|--------|------|-------------|
| `Male_15_19` | `int64` | Count Gender b/t 15-19 years old|
| `Male_20_24` | `int64` | Count Gender b/t 20-24 years old|
| `Male_25_29` | `int64` | Count Gender b/t 25-29 years old|
| `Male_30_34` | `int64` | Count Gender b/t 30-34 years old|
| `Male_35_39` | `int64` | Count Gender b/t 35-39 years old|
| `Male_40_44` | `int64` | Count Gender b/t 40-44 years old|
| `Male_45_49` | `int64` | Count Gender b/t 45-49 years old|
| `Male_50_54` | `int64` | Count Gender b/t 50-54 years old|
| `Male_55_59` | `int64` | Count Gender b/t 55-59 years old|
| `Male_60_` | `int64`   | Count Gender over 60+ years old|
| `Female_15_19` | `int64`| Count Gender b/t 15-19 years old| 
| `Female_20_24` | `int64`| Count Gender b/t 20-24 years old|
| `Female_25_29` | `int64`| Count Gender b/t 25-29 years old|
| `Female_30_34` | `int64`| Count Gender b/t 30-34 years old|
| `Female_35_39` | `int64`| Count Gender b/t 35-39 years old
| `Female_40_44` | `int64`| Count Gender b/t 40-44 years old|
| `Female_45_49` | `int64`| Count Gender b/t 45-49 years old|
| `Female_50_54` | `int64`| Count Gender b/t 50-54 years old|
| `Female_55_59` | `int64`| Count Gender b/t 55-59 years old|
| `Female_60_` | `int64` | Count Gender over 60+ years old|
| `MedianHHIncome` | `int64` | Median HouseHold Income |
| `HHBelowPovery` | `float64` | Households below Poverty |
| `OwnerOccupy` | `int64` | Owner lives in house |
| `OwnerOccupyPct` | `float64` | Owner lives in house % |
| `RenterOccupy` | `int64` | Rental Property |
| `RenterOccupyPct` | `float64` | Rental Property % |
| `VacantHU` | `int64` | Vacant Housing Units |
| `VacantHUPct` | `float64` | Vacant Housing Units % |
| `HU_1_Detached` | `int64` | No description available |
| `HU_2` | `int64` | No description available |
| `HU_3_4` | `int64` | No description available |
| `HU_5_9` | `int64` | No description available |
| `HU_10_19` | `int64` | No description available |
| `HU_20_49` | `int64` | No description available |
| `HU_50_` | `int64` | No description available |
| `WorkHome` | `float64` | Work From Home |
| `PublicTrans` | `float64` | Use Public Transit |
| `DroveAlone` | `float64` | Drive Alone |
| `Carpooled` | `float64` | Carpool |
| `Walked` | `float64` | Walk to Work |
| `Bicycle` | `float64` | Bike to Work |
| `OtherMeans` | `float64` | Other Commute |
| `MedianAge` | `float64` | Median Age |
| `TotalPop` | `int64` | Total Population |
| `ACRES` | `float64` | Size in Acres |
| `SQMILES` | `float64` | Size in Sq. Miles |


### Drivetimes
- **Description**: 5, 10, 15 minute drivetimes from every Starbucks location in Atlanta
- **Source**: ArcGIS Pro
- **Rows**: 177
- **Columns**:

| Column | Type | Description |
|--------|------|-------------|
| `OBJECTID` | `int64` | Unique row identifier |
| `OBJECTID_1` | `int64` | No description available |
| `AREA_ID` | `object` | Area ID |
| `AREA_DESC` | `object` | No description available |
| `Drive_Time` | `object` | No description available |
| `Store_Numb` | `object` | Store number (internal ID) |
| `Shape_Leng` | `float64` | No description available |
| `Shape_Length` | `float64` | No description available |
| `Shape_Area` | `float64` | No description available |


### We can now load the data....

In [2]:
drivetimes_gdf = gpd.read_file("data/geojson_data/drivetimes.geojson")
demographics_gdf = gpd.read_file("data/geojson_data/demographics.geojson")
starbucks_gdf = gpd.read_file("data/geojson_data/starbucks_locations.geojson")
poi_gdf = gpd.read_file("data/geojson_data/poi_data.geojson")
gdot_gdf = gpd.read_file("data/geojson_data/gdot_traffic.geojson")
competition_gdf = gpd.read_file("data/geojson_data/competition.geojson")

In [3]:
# filters competitors down to only the brands that serve beverages or small snacks similar to Starbucks
def filter_competitor_brands(gdf, brand_list, brand_column="brand"):

    filtered = gdf[gdf[brand_column].isin(brand_list)]
    print(f"Filtered to {len(filtered)} matching competitor locations.")
    return filtered

brand_list = [
"McDonald's",
"Caribou Coffee",
"The Coffee Bean & Tea Leaf",
"Chick-fil-A",
"Einstein Bros. Bagels",
"Wendy's",
"Dunkin'",
"Burger King",
"Smoothie King",
"Dunkin' Donuts",
"Kung Fu Tea",
"Corner Bakery",
"Panera Bread",
"Krispy Kreme",
"Tim Hortons",
"Jamba"
]

# Example usage
competition_gdf = filter_competitor_brands(competition_gdf, brand_list)

Filtered to 149 matching competitor locations.


## Get Nearest Traffic Counts
We will now join each Starbucks location to the nearest traffic count.

In [4]:
def get_nearest_traffic(starbucks_gdf, traffic_gdf):
    # Filter GDOT traffic stations to include only those with AADT > 0
    traffic_gdf = traffic_gdf[traffic_gdf["AADT"] > 0]
    # Project both datasets to a metric CRS for accurate distance calculations
    starbucks_proj = starbucks_gdf.to_crs(epsg=3857)
    traffic_proj = traffic_gdf.to_crs(epsg=3857)

    # Find nearest traffic count
    traffic_counts = []
    for star_geom in starbucks_proj.geometry:
        distances = traffic_proj.geometry.distance(star_geom)
        nearest_idx = distances.idxmin()
        aadt_value = traffic_proj.loc[nearest_idx, "AADT"]
        traffic_counts.append(aadt_value)

    # Add result to original Starbucks GeoDataFrame
    starbucks_gdf["traffic_count"] = traffic_counts
    return starbucks_gdf

In [5]:
# Apply the function
starbucks_gdf = get_nearest_traffic(starbucks_gdf, gdot_gdf)

# Preview
starbucks_gdf[["Store_Name", "traffic_count"]].head()


,Store_Name,traffic_count
0,Kroger Atlanta #672,7500
1,Spelman University Manley Student C,1180
2,The Hurt Building,16600
3,AmericasMart,14600
4,Embassy Suites - Centennial Park,14600


# Enriching the Datasets

#### What is enrichment?
Enrichment is basically a 'join' where we join the input features (in this case, starbucks locations) with surrounding information. We will be joining the starbucks locations with competitor counts in 5 minutes of driving, poi data within 5 minutes of driving, and our demographic information. This should give us a fairly large dataset that we can determine our attractiveness on.

In [6]:
def enrich_starbucks_with_competitor_counts(starbucks_gdf, drivetimes_gdf, competition_gdf, drive_time_value="5"):
    filtered_gdf = drivetimes_gdf[drivetimes_gdf["Drive_Time"] == drive_time_value].reset_index(drop=True)

    results = []
    for _, row in filtered_gdf.iterrows():
        polygon = row.geometry
        # here we are going through the 5 mile drive time polygons and determining which starbucks are within that polygon area.
        count = competition_gdf[competition_gdf.geometry.within(polygon)].shape[0]
        # we then take the store number and determine the count of features within that area.
        results.append({
            "Store_Numb": row["Store_Numb"],
            "competitor_count": count
        })

    counts_df = pd.DataFrame(results)

    # Prevent duplicate column merge
    starbucks_gdf = starbucks_gdf.drop(columns=["competitor_count"], errors="ignore")

    enriched = pd.merge(starbucks_gdf, counts_df, on="Store_Numb", how="left")
    
    return enriched


In [7]:
def enrich_starbucks_with_poi_counts(starbucks_gdf, drivetimes_gdf, poi_gdf, drive_time_value="5"):
    school_keywords = ["school", "college", "university"]
    hospital_keywords = ["hospital", "clinic"]
    supermarket_keywords = ["supermarket", "grocery", "convenience"]

    filtered_gdf = drivetimes_gdf[drivetimes_gdf["Drive_Time"] == drive_time_value].reset_index(drop=True)
    poi_gdf = poi_gdf.to_crs(filtered_gdf.crs)

    # here we are going through the 5 mile drive time polygons and determining which starbucks are within that polygon area.
    results = []
    for _, row in filtered_gdf.iterrows():
        polygon = row.geometry
        pois_in_poly = poi_gdf[poi_gdf.geometry.within(polygon)]
        # we then take the store number and determine the count of features within that area.
        results.append({
            "Store_Numb": row["Store_Numb"],
            "school_count": pois_in_poly["fclass"].str.contains("|".join(school_keywords), case=False, na=False).sum(),
            "hospital_count": pois_in_poly["fclass"].str.contains("|".join(hospital_keywords), case=False, na=False).sum(),
            "supermarket_count": pois_in_poly["fclass"].str.contains("|".join(supermarket_keywords), case=False, na=False).sum()
        })

    poi_df = pd.DataFrame(results)

    # Drop to avoid _x/_y suffixes
    starbucks_gdf = starbucks_gdf.drop(columns=[
        "school_count", "hospital_count", "supermarket_count"
    ], errors="ignore")

    enriched = pd.merge(starbucks_gdf, poi_df, on="Store_Numb", how="left")


    return enriched

In [8]:
def join_starbucks_with_demographics(starbucks_gdf, demographics_gdf):
    demo_attrs_list = []

    for pt in starbucks_gdf.geometry:
        match = None
        for _, row in demographics_gdf.iterrows():
            if pt.within(row.geometry):
                match = row.drop(labels="geometry").to_dict()
                break
        demo_attrs_list.append(match if match is not None else {})

    demo_df_expanded = pd.DataFrame(demo_attrs_list)

    # Drop any conflicting columns
    overlapping_cols = demo_df_expanded.columns.intersection(starbucks_gdf.columns)
    starbucks_gdf = starbucks_gdf.drop(columns=overlapping_cols, errors="ignore")

    result = pd.concat([starbucks_gdf.reset_index(drop=True), demo_df_expanded.reset_index(drop=True)], axis=1)
    return result

In [9]:
# Enrich with competitor counts
starbucks_gdf = enrich_starbucks_with_competitor_counts(
    starbucks_gdf, drivetimes_gdf, competition_gdf, drive_time_value="5"
)

# Enrich with schools, hospitals, supermarkets
starbucks_gdf = enrich_starbucks_with_poi_counts(starbucks_gdf, drivetimes_gdf, poi_gdf, drive_time_value="5")

# Enrich Starbucks dataset with demographics
starbucks_gdf = join_starbucks_with_demographics(starbucks_gdf, demographics_gdf)

# set all numeric columns to float and then make all na 0.0
starbucks_gdf[starbucks_gdf.select_dtypes(include='number').columns] = starbucks_gdf.select_dtypes(include='number').astype(float)
starbucks_gdf = starbucks_gdf.fillna(0.0)

# Preview final result
starbucks_gdf[[
    "Store_Name", "traffic_count", "competitor_count",
    "school_count", "hospital_count", "supermarket_count"
]].head()

,Store_Name,traffic_count,competitor_count,school_count,hospital_count,supermarket_count
0,Kroger Atlanta #672,7500.0,0.0,6.0,0.0,1.0
1,Spelman University Manley Student C,1180.0,4.0,13.0,0.0,0.0
2,The Hurt Building,16600.0,8.0,16.0,0.0,0.0
3,AmericasMart,14600.0,10.0,8.0,0.0,0.0
4,Embassy Suites - Centennial Park,14600.0,10.0,8.0,0.0,0.0


# Determining Attractiveness
### Doing a Principal Component Analysis

Here I am going to get all of the numeric data columns we got from enriching our data. We will then use these columns to do a Principal Component Analysis to determine our important data for creating our Huff Model. 

Recall that are Huff Model is some attractiveness divided by the distance decay. We need to determine here what makes a store attractice and how to weigh each factor!

In [10]:
# gets list of columns to run PCA on
def get_pca_columns(gdf):
    cols = gdf.columns.tolist()

    # Indices of enrichment columns
    start_enrich = cols.index("traffic_count")
    end_enrich = cols.index("supermarket_count") + 1

    # Indices of demographic slice
    start_demo = cols.index("TotalPop")
    end_demo = cols.index("Carpooled") + 1

    # Slice and combine
    candidate_cols = cols[start_enrich:end_enrich] + cols[start_demo:end_demo]

    # Filter to keep only numeric columns
    numeric_cols = gdf.select_dtypes(include='number').columns
    pca_cols = [col for col in candidate_cols if col in numeric_cols]

    return pca_cols

### Here we are calculating our attractiveness score using PCA!

In [11]:
def compute_attractiveness_score(df, feature_cols=None):
    if feature_cols is None:
        feature_cols = [
            "traffic_count", "competitor_count",
            "school_count", "hospital_count", "supermarket_count"
        ]

    # Standardize features
    X = df[feature_cols].fillna(0)  # Ensure no NaNs
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Perform PCA
    pca = PCA(n_components=1)
    principal_component = pca.fit_transform(X_scaled)

    # Use first component loadings as weights
    weights = pca.components_[0]
    
    # Save weights if you'd like to inspect them
    weight_dict = dict(zip(feature_cols, weights))
    print("PCA Weights:", weight_dict)

    # Compute attractiveness using weighted sum
    df["attractiveness"] = X_scaled @ weights
    
    # Shift the entire column to ensure all values are positive
    df["attractiveness"] = df["attractiveness"] - df["attractiveness"].min() + 1e-6

    return df


## Running our attractiveness scores!

Below I am getting the columns I want to run my PCA on and the weights will be displayed. 
Then, I will compute the attractiveness scores. 

In [12]:
# Get the columns that we want to run the PCA on
pca_cols = get_pca_columns(starbucks_gdf)
# Calculate the Attractiveness Scores
starbucks_gdf = compute_attractiveness_score(starbucks_gdf, pca_cols)

PCA Weights: {'traffic_count': 0.05299559990667477, 'competitor_count': -0.1099214250212515, 'school_count': -0.054346081025821974, 'hospital_count': -0.08347296233472516, 'supermarket_count': 0.056852721728809834, 'TotalPop': -0.13908404673004807, 'MedianAge': 0.09174391157558172, 'MalePop': -0.1389952879508659, 'MalePopPct': -0.1213194155292017, 'Male_0_4': -0.13011360382517384, 'Male_0_4Pct': 0.08227164561394726, 'Male_5_9': -0.11036419716434108, 'Male_5_9Pct': 0.09496467157588515, 'Male_10_14': -0.0869279362405235, 'Male_10_14Pct': 0.09021695164179809, 'Male_15_19': -0.07933810711095317, 'Male_15_19Pct': -0.0425460327040381, 'Male_20_24': -0.10980318778306282, 'Male_20_24Pct': -0.06338726843192816, 'Male_25_29': -0.12565167623772772, 'Male_25_29Pct': -0.04120584966327489, 'Male_30_34': -0.12817235181667247, 'Male_30_34Pct': -0.025917090396376245, 'Male_35_39': -0.12841400340159786, 'Male_35_39Pct': -0.019727424888994257, 'Male_40_44': -0.1355249029280403, 'Male_40_44Pct': 0.0316009

In [13]:
# Preview
starbucks_gdf[["Store_Name", "attractiveness"]].head()

,Store_Name,attractiveness
0,Kroger Atlanta #672,13.735181
1,Spelman University Manley Student C,15.760780
2,The Hurt Building,4.593737
3,AmericasMart,4.668575
4,Embassy Suites - Centennial Park,4.668575


## Computing the distance matrix

Next we will calculate the distance from each store to a neighborhood, but we need to determine where to measure the distance. Here we are computing a distance matrix and using the centroid of each neighborhood as a point of reference. This will be used as our distance in our distance weight in the Huff Model.

In [14]:
def compute_distance_matrix(demographics_gdf, starbucks_gdf):
    # Compute centroids of demographic polygons
    demo_centroids = demographics_gdf.geometry.centroid

    # Reproject both to a metric CRS (Web Mercator EPSG:3857)
    stores_proj = starbucks_gdf.to_crs(epsg=3857)
    centroids_proj = demo_centroids.to_crs(epsg=3857)

    # Compute distance matrix (each row = demo zone, each col = store)
    distance_matrix = np.array([
        centroids_proj.distance(store_geom) for store_geom in stores_proj.geometry
    ]).T  # shape: (zones x stores)
    
    # Convert to miles
    distance_matrix = distance_matrix / 1609.34

    return distance_matrix, centroids_proj

### Ignore the warning below


In [15]:
distance_matrix, demo_centroids = compute_distance_matrix(demographics_gdf, starbucks_gdf)

<ipython-input-14-18cc69b54432>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  demo_centroids = demographics_gdf.geometry.centroid


# Compute our Huff Probabilities
Recall that a Huff Model is basically the adjusted attractiveness of a store by distance divided by the sum of all adjusted attractiveness:

$$
P_{ij} = \frac{A_j^\lambda / D_{ij}^\beta}{\sum_{k=1}^{n} A_k^\lambda / D_{ik}^\beta}
$$

Where:
- $P_{ij}$: Probability that consumer in zone $i$ visits store $j$
- $A_j$: Attractiveness of store $j$
- $D_{ij}$: Distance (or travel time) from zone $i$ to store $j$
- $\beta$: Distance decay parameter (typically > 1)
- $n$: Total number of competing stores


In [16]:
def compute_huff_probabilities(distance_matrix, attractiveness, decay=1):
    # we set the distance matrix to 1 when it is 0 to avoid dividing by zero.
    safe_distances = np.where(distance_matrix == 0, 1, distance_matrix)
    # for each store and neighborhood, we are adjusting the attractiveness by a particular distance (attractiveness/dist.)
    numerators = attractiveness / np.power(safe_distances, decay)
    # for each store/neighborhood adjusted by distance, we are taking the sum 
    denominators = numerators.sum(axis=1, keepdims=True)
    # the huff model is the ratio between the adjusted attractiveness divided by the sum of all the adjusted attractiveness
    probabilities = numerators / denominators
    return probabilities



In [17]:
# Compute probabilities
attractiveness = starbucks_gdf["attractiveness"].values
huff_probs = compute_huff_probabilities(distance_matrix, attractiveness)

# Show our shape
print("Huff matrix shape:", huff_probs.shape)  # (zones, stores)


Huff matrix shape: (248, 34)


# Visualization

Now we will create an **interactive** heatmap to show which neighborhoods the customers are most likely to visit from. We are using the folium package here to create a map. We can also use matplotlib to create the heatmap with blue being the less likely areas to visit a store and red being the most likely. Below the create_heatmap cell, we are calling on a random store to see it's Huff Probabilities. 

**Feel free to keep loading that cell to see different stores and hover and click on the map to learn about the area!**

In [18]:
def create_heatmap(store_number, huff_probs, starbucks_gdf, demographics_gdf, icon_path=None, label_field="NAME"):
    # Get index of the selected store
    store_index_list = starbucks_gdf.index[starbucks_gdf["Store_Numb"] == store_number].tolist()
    store_idx = store_index_list[0]

    # Pull zone probabilities and transform
    zone_probs = huff_probs[:, store_idx]
    transformed_probs = np.sqrt(zone_probs)
    vmax = np.percentile(transformed_probs, 95)
    norm = colors.Normalize(vmin=0, vmax=vmax)
    cmap = cm.get_cmap("coolwarm")

    # Center map on the selected store
    store_point = starbucks_gdf.loc[store_idx, "geometry"]
    m = folium.Map(location=[store_point.y, store_point.x], zoom_start=11)

    # Iterate through demographics GeoDataFrame
    for geom, prob, transformed, props in zip(
        demographics_gdf.geometry, zone_probs, transformed_probs, demographics_gdf.to_dict("records")
    ):
        rgba = cmap(norm(transformed))
        hex_color = colors.to_hex(rgba)
        name = props.get(label_field, "Unknown")

        folium.GeoJson(
            data={
                "type": "Feature",
                "geometry": mapping(geom),  # ensure GeoJSON compatibility
                "properties": {
                    "probability": str(round(prob*100, 3)) + ' %',
                    "name": name
                }
            },
            style_function=lambda feature, color=hex_color: {
                "fillColor": color,
                "color": "gray",
                "weight": 0.4,
                "fillOpacity": 0.7
            },
            tooltip=folium.GeoJsonTooltip(fields=["name", "probability"],
                                          aliases=["Neighborhood", "Huff Probability"])
        ).add_to(m)

    # Add store marker
    if icon_path:
        icon = CustomIcon(icon_path, icon_size=(20, 20))
        folium.Marker([store_point.y, store_point.x], icon=icon,
                      popup=starbucks_gdf.loc[store_idx, "Store_Name"]).add_to(m)
    else:
        folium.Marker([store_point.y, store_point.x],
                      popup=starbucks_gdf.loc[store_idx, "Store_Name"]).add_to(m)

    return m

## Keep Running the cell over and over to roll a new Starbuck's Location!

In [19]:
# Randomly select a store number from the GeoDataFrame
random_store = np.random.choice(starbucks_gdf["Store_Numb"])

# Generate the heatmap for that store
create_heatmap(
    store_number=random_store,
    huff_probs=huff_probs,
    starbucks_gdf=starbucks_gdf,
    demographics_gdf=demographics_gdf,
    icon_path="data/icons/starbucks_logo.png"
)

# Results and Conclusions

## General Observations
From this Huff model analysis, we observe that the probabilities of customers visiting each Starbucks location are mostly intuitive and realistic. One clear trend is the decrease in individual store probabilities in the downtown core of Atlanta, which aligns with expectations — downtown areas have a higher density of Starbucks locations, meaning each individual store is competing with many others for the same pool of customers. This naturally dilutes the probability for any one store in that area.

Another notable insight is that the Downtown neighborhood consistently appears with low Huff probabilities (often visualized in cooler or blue tones on the map). This is likely because this zone consists primarily of commercial buildings, office towers, and public spaces, and thus has fewer residential consumers compared to surrounding zones. Since the Huff model distributes probabilities based on residential centroids (where consumers are assumed to live), zones with minimal population will inherently contribute little to any store’s probability distribution. For future modeling, excluding or flagging non-residential zones could enhance model clarity and prevent misleading cold spots in the visualizations.

It’s also worth noting that color variation across zones can appear limited, especially when visualized on a heatmap. This could be due to:

The scaling of the colormap, where a few dominant values skew the gradient, making subtle differences harder to detect.

Or the lack of variance in input attractiveness scores (e.g., all stores being similarly attractive), which leads to a flatter probability distribution across zones.

Despite these limitations, the core question of identifying where customers are most likely coming from has been addressed. The Huff model gives us a spatially grounded, probability-based view of consumer behavior, useful for both strategic and operational decisions.

## What Can We Do with This Data?
This analysis opens up several practical applications:

Targeted Marketing & Promotions
Areas with low Huff probabilities could be targeted with local marketing campaigns to increase awareness and pull. Conversely, areas with high probabilities could receive loyalty programs or special offers to retain existing strong customer bases.

Site Selection for New Stores
Areas where probabilities are high but existing Starbucks locations are far away or nonexistent could be ideal candidates for new store development. This approach turns probabilistic demand into actionable expansion strategies.

Enhanced Customer Segmentation & Modeling
By combining Huff probabilities with actual sales or foot traffic data, we can refine the attractiveness score calculations and validate assumptions. Over time, this could evolve into a machine-learning-based customer behavior model grounded in real outcomes.

Operational Planning
Stores with overlapping trade areas may experience internal competition. Understanding these overlaps can inform decisions about store differentiation, inventory allocation, or menu specialization to serve segmented demand more efficiently.